# A Simple Guide of CoolBox


In [ ]:
# import packages

from coolbox.api import *

## Download some example files

Firstly we download some example files from public database. Here, all of our data is about the K562 cancer cell line.

In [1]:
%%bash

mkdir data

# example .cool file
wget -O data/K562_MbolI_5kb.cool ftp://cooler.csail.mit.edu/coolers/hg19/Rao2014-K562-MboI-allreps-filtered.5kb.cool
# .looplist file
wget -O data/K562_MbolI_looplist.txt.gz ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE63nnn/GSE63525/suppl/GSE63525_K562_HiCCUPS_looplist.txt.gz

# bigwig files (ChIP-Seq and RNA-Seq data)
wget -O data/K562_H3K27ac.bigWig  ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM733nnn/GSM733656/suppl/GSM733656_hg19_wgEncodeBroadHistoneK562H3k27acStdSig.bigWig
wget -O data/K562_H3K27me3.bigWig ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM733nnn/GSM733658/suppl/GSM733658_hg19_wgEncodeBroadHistoneK562H3k27me3StdSig.bigWig
wget -O data/K562_H3K4me3.bigWig  ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM945nnn/GSM945297/suppl/GSM945297_hg19_wgEncodeUwHistoneK562H3k04me3StdZnf2c10c5RawRep1.bigWig
wget -O data/K562_RNASeq.bigWig ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM765nnn/GSM765393/suppl/GSM765393_wgEncodeCshlLongRnaSeqK562NucleolusTotalMinusRawSigRep4.bigWig
# ChIA-PET data
wget -O data/K562_chiapet_interaction.txt.gz ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM832nnn/GSM832465/suppl/GSM832465_CHK019M_L2_lane24.ChromatinInteractions.bed.gz
wget -O data/K562_chiapet.bigWig ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM832nnn/GSM832465/suppl/GSM832465_CHK019M_L2_lane24.bw

# RefSeq
wget -O data/refGene.txt.gz http://hgdownload.soe.ucsc.edu/goldenPath/hg19/database/refGene.txt.gz

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
!ls -lh data

total 2.3G
-rw-rw-r-- 1 nanguage nanguage  81M 4月   9 17:32 K562_chiapet.bigWig
-rw-rw-r-- 1 nanguage nanguage 2.0M 4月   9 17:31 K562_chiapet_interaction.txt.gz
-rw-rw-r-- 1 nanguage nanguage 293M 4月   9 16:56 K562_H3K27ac.bigWig
-rw-rw-r-- 1 nanguage nanguage 384M 4月   9 17:13 K562_H3K27me3.bigWig
-rw-rw-r-- 1 nanguage nanguage  83M 4月   9 17:17 K562_H3K4me3.bigWig
-rw-rw-r-- 1 nanguage nanguage 528M 4月   9 16:38 K562_MbolI_5kb.cool
-rw-rw-r-- 1 nanguage nanguage 363K 4月   9 16:53 K562_MbolI_looplist.txt.gz
-rw-rw-r-- 1 nanguage nanguage 926M 4月   9 17:31 K562_RNASeq.bigWig
-rw-rw-r-- 1 nanguage nanguage 6.5M 4月   8 21:50 refGene.txt.gz
